<a href="https://colab.research.google.com/github/imabari/ehime-covid-bot/blob/main/ehime_covid_main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pdfplumber

In [ ]:
# GASのウェブアプリのURLを登録
GAS_WEBAPPS = ""

In [ ]:
import datetime
import pathlib
import re
import unicodedata

In [ ]:
import json
import pdfplumber
import requests

In [ ]:
def fetch_file(url, dir="."):

    p = pathlib.Path(dir, pathlib.PurePath(url).name)
    p.parent.mkdir(parents=True, exist_ok=True)

    r = requests.get(url)

    with p.open(mode="wb") as fw:
        fw.write(r.content)

    return p

In [ ]:
p = fetch_file("https://www.pref.ehime.jp/h25500/kansen/documents/kennai_link.pdf")

In [ ]:
pdf = pdfplumber.open(p)

In [ ]:
JST = datetime.timezone(datetime.timedelta(hours=+9), "JST")
dt_now = datetime.datetime.now(JST).replace(tzinfo=None)

In [ ]:
year = dt_now.year

for page in pdf.pages[::-1]:

    s = unicodedata.normalize("NFKC", page.extract_text())

    if s.startswith("県内における新型コロナウイルス感染症患者の発生状況について"):

        m = re.search("(\d{1,2})月(\d{1,2})日 +(\d{1,2})時現在", s)
        month, day, hour = map(int, m.groups())

        dt_date = datetime.datetime(year, month, day, hour)

        if dt_date > dt_now:
            dt_date = datetime.datetime(year - 1, month, day, hour)

        data = [int(i.replace(",", "")) for i in re.findall("([0-9,]+)人", s)]

        print(dt_date, data)

        if len(data) == 9:

            url = GAS_WEBAPPS
            headers = {"Content-Type": "application/json"}

            json_data = json.dumps({"data": [dt_date.isoformat()] + data})
            requests.post(url, json_data, headers=headers)

        break